In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import DepthwiseConv2D

# Create a custom DepthwiseConv2D class that ignores the 'groups' argument
class CustomDepthwiseConv2D(DepthwiseConv2D):
    def __init__(self, *args, **kwargs):
        if 'groups' in kwargs:
            kwargs.pop('groups')  # Remove 'groups' argument
        super().__init__(*args, **kwargs)

# Custom Classifier class
class Classifier:
    def __init__(self, modelPath, labelsPath):
        self.model = load_model(modelPath, custom_objects={'DepthwiseConv2D': CustomDepthwiseConv2D})
        self.labels = self.load_labels(labelsPath)

    def load_labels(self, labelsPath):
        with open(labelsPath, 'r') as f:
            return [line.strip() for line in f.readlines()]

    def getPrediction(self, img, draw=True):
        img = cv2.resize(img, (224, 224))  # Adjust size if needed
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)
        
        prediction = self.model.predict(img)
        index = np.argmax(prediction)
        
        return prediction[0], index

# Initialize components
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier(r"C:\Users\sp_kr\OneDrive\Desktop\Project_Sigma\converted_keras\keras_model.h5",
                        r"C:\Users\sp_kr\OneDrive\Desktop\Project_Sigma\converted_keras\labels.txt")

offset = 20
imgSize = 300
labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "space"]

while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[max(0, y-offset):min(img.shape[0], y+h+offset), 
                      max(0, x-offset):min(img.shape[1], x+w+offset)]
        
        aspectRatio = h / w
        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
        
        prediction, index = classifier.getPrediction(imgWhite, draw=False)
        
        cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)
        
        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)
    
    cv2.imshow("Image", imgOutput)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()